In [1]:
## Inicialização dos modelos
using DataFrames
using CSV
using Dates
ENV["lines"]=200

200

In [2]:
function chercher(DF,palavra)
    #funcao que para dado um df coluna, retorna a posicao das palavras escolhidas
    position=[]
    for i in range(1,size(DF)[1],step=1)
        if DF[i] == palavra
            append!( position, i )
        end
    end
    return position
end


function match(array1,array2)
    #Funcao que dado dois arrays de posicoes (vindos do chercher), retorna as posiçoes que sao comuns aos dois arrays
    multiple=[]
    for i in range(1,size(array1)[1],step=1)
        for j in range(1,size(array2)[1],step=1)
            if array1[i]==array2[j]
                append!( multiple, array1[i] )
            end
        end
    end
    return multiple
end

match (generic function with 1 method)

In [3]:
study_path = "C:/Users/lucas/Dropbox (Compte personnel)/Faculdade/EstagioPSR/FORTSCUE/00-CasoBase/2025/"; #raiz onde ficará todos os casos
ano = "2025"
lines = DataFrame(CSV.File(study_path*ano*"Lines.csv",header=4,delim=";"));
trafo = DataFrame(CSV.File(study_path*ano*"Transfo.csv",header=4,delim=";"));


In [4]:
#Defini-se aqui as barras as quais buscaremos as vizinhanças e também as Areas de interesse do estudo (ideal por todas as areas que temos elementos até a 3 viz)
bus = ["PECEM2-CE500"]
area = [3,118, 7, 4, 2, 102]
#bus = ["PECEM2-CE230"]

6-element Vector{Int64}:
   3
 118
   7
   4
   2
 102

In [5]:
#Criação de headers para o dataframe das vizinhanças
nome=["From#","From Name","To# - Circ#","To Name","Area"] 
viz1=DataFrame()
viz2=DataFrame()
viz3=DataFrame()
for i in range(1,length(nome),step=1)
    if i == 1
        viz1[!,nome[i]]=Int[]
        viz2[!,nome[i]]=Int[]
        viz3[!,nome[i]]=Int[]
    else
        viz1[!,nome[i]]=String[]
        viz2[!,nome[i]]=String[]
        viz3[!,nome[i]]=String[]
    end
end

# Funções gerais de contigência para qualquer vizinhança

In [6]:
function cont(Surface,Bus,saida)
    #Função que busca as linhas e transformadores na 1 vizinhança das barras fornecidas, na area desejada.
    #Surface = Lista de áreas de interesse
    #Bus = Lista de barras que buscaremos as vizinhanças diretas
    #Saida = DataFrame onde será salvo os dados
    
    
    LinhasArea=[]
    TrafoArea=[]
    for a in Surface #Para filtrar o arquivo de linhas e transfo pela aréa igual a buscada
        LinhasArea=vcat(LinhasArea,chercher(lines[!,"Area"],a))
        TrafoArea=vcat(TrafoArea,chercher(trafo[!,"Area"],a))
    end
    #####
    
    
    elemL=[]
    elemT=[]
    for b in Bus #para determinar quais linhas possuem a barra saindo ou chegando com os nomes desejados
        elemL=vcat(elemL,chercher(lines[!,"From Name"],b))
        elemL=vcat(elemL,chercher(lines[!,"To Name"],b))

        elemT=vcat(elemT,chercher(trafo[!,"From Name"],b))
        elemT=vcat(elemT,chercher(trafo[!,"To Name"],b))
    end
    ####
    #para fazer o match cruzado entre as que estão na area desejada e as que possuem o nome de partida/chegada desejada
    Linhas = match(elemL,LinhasArea)
    Trafo =match(elemT,TrafoArea)
    
    for i in Linhas #adicionar no arquivo de saida as contingencias
        col1=lines[i,1]
        col2=lines[i,2]
        col3=lines[i,3]
        col4=lines[i,4]
        col5=string(lines[i,5])
        push!(saida,[col1,col2,col3,col4,col5])
    end

    for i in Trafo
        col1=trafo[i,1]
        col2=trafo[i,2]
        col3=trafo[i,3]
        col4=trafo[i,4]
        col5=string(trafo[i,5])
        push!(saida,[col1,col2,col3,col4,col5])
    end
    
    return saida
    
end

cont (generic function with 1 method)

In [7]:
function outOrganon(vizinhanca)
    #Função que retorna em formato texto o arquivo de contingencia pronto para ser posto no organon. Dessa vez exclui-se circuitos duplos ou mais
    #Vizinhanca = dataframe produzido pela função cont
    
    for i in range(1,size(vizinhanca)[1],step=1)
        partida=string(vizinhanca[i,1])
        indc=findfirst(isequal(' '), vizinhanca[i,3])
        chegada=vizinhanca[i,3][1:indc-1]
        
        indcir=findfirst(isequal('#'), vizinhanca[i,3])
        circ=vizinhanca[i,3][indcir+2:end]
        ncirc=parse(Int,circ)
        if ncirc>1 #pular segundos circuitos
            continue
        end
        
        
        
    
        namefrom=vizinhanca[i,2]
        nameto=vizinhanca[i,4]
        

        println("'"*namefrom*"_"*nameto*"\t'")
        println("BRANCH\t"*partida*"\t"*chegada*"\t01")
        println("\tEND /")
    end
     println("END /")
end

outOrganon (generic function with 1 method)

# Contingência 1 vizinhança

In [8]:
cont(area,bus,viz1)

,From#,From Name,To# - Circ#,To Name,Area
,Int64,String,String,String,String
1,11525,PECEM2-CE500,525 # 02,FORTA2-CE500,3
2,526,SOBRA3-CE500,11525 # 01,PECEM2-CE500,3
3,526,SOBRA3-CE500,11525 # 02,PECEM2-CE500,3
4,822,PACATB-CE500,11525 # 01,PECEM2-CE500,3
5,12902,ACARA3-CE500,11525 # 01,PECEM2-CE500,3
6,11225,PECEM2-CE230,11525 #01,PECEM2-CE500,3
7,11225,PECEM2-CE230,11525 #02,PECEM2-CE500,3
8,11225,PECEM2-CE230,11525 #03,PECEM2-CE500,3


In [9]:
outOrganon(viz1)

'SOBRA3-CE500_PECEM2-CE500	'
BRANCH	526	11525	01
	END /
'PACATB-CE500_PECEM2-CE500	'
BRANCH	822	11525	01
	END /
'ACARA3-CE500_PECEM2-CE500	'
BRANCH	12902	11525	01
	END /
'PECEM2-CE230_PECEM2-CE500	'
BRANCH	11225	11525	01
	END /
END /


# Contingência 2 vizinhança

In [15]:
cont2=[]
for i in range(1,size(viz1)[1],step=1)
    if viz1[i,2] in bus
        name=viz1[i,4]
    else 
        name=viz1[i,2]
    end
    cont2=vcat(cont2,name)
end
cont2=unique(cont2);

In [16]:
cont(area,cont2,viz2)

,From#,From Name,To# - Circ#,To Name,Area
,Int64,String,String,String,String
1,525,FORTA2-CE500,822 # 01,PACATB-CE500,3
2,516,QUIXAD-CE500,525 # 01,FORTA2-CE500,3
3,11525,PECEM2-CE500,525 # 02,FORTA2-CE500,3
4,526,SOBRA3-CE500,11525 # 01,PECEM2-CE500,3
5,526,SOBRA3-CE500,11525 # 02,PECEM2-CE500,3
6,12905,TIANG2-CE500,526 # 01,SOBRA3-CE500,3
7,12905,TIANG2-CE500,526 # 02,SOBRA3-CE500,3
8,822,PACATB-CE500,11525 # 01,PECEM2-CE500,3
9,822,PACATB-CE500,56100 # 01,JAGUA2-CE500,3


In [17]:
outOrganon(viz2)

'FORTA2-CE500_PACATB-CE500	'
BRANCH	525	822	01
	END /
'QUIXAD-CE500_FORTA2-CE500	'
BRANCH	516	525	01
	END /
'SOBRA3-CE500_PECEM2-CE500	'
BRANCH	526	11525	01
	END /
'TIANG2-CE500_SOBRA3-CE500	'
BRANCH	12905	526	01
	END /
'PACATB-CE500_PECEM2-CE500	'
BRANCH	822	11525	01
	END /
'PACATB-CE500_JAGUA2-CE500	'
BRANCH	822	56100	01
	END /
'FORTA2-CE500_PACATB-CE500	'
BRANCH	525	822	01
	END /
'ACARA3-CE500_PECEM2-CE500	'
BRANCH	12902	11525	01
	END /
'PARNAI-PI500_ACARA3-CE500	'
BRANCH	12901	12902	01
	END /
'TIANG2-CE500_ACARA3-CE500	'
BRANCH	12905	12902	01
	END /
'PECEM2-CE230_CFAISA-CE230	'
BRANCH	11225	55777	01
	END /
'PECEM2-CE230_CTAIBA-CE230	'
BRANCH	11225	55784	01
	END /
'PECEM2-CE230_TRAIRI-CE230	'
BRANCH	11225	55790	01
	END /
'CSP----CE230_PECEM2-CE230	'
BRANCH	316	11225	01
	END /
'ENPECE-CE230_PECEM2-CE230	'
BRANCH	511	11225	01
	END /
'SMONIC-PI230_PECEM2-CE230	'
BRANCH	55798	11225	01
	END /
'SERROT-CE230_PECEM2-CE230	'
BRANCH	55814	11225	01
	END /
'FORTA2-CE230_FORTA2-CE500	'
BRANCH	32

#  Contingência 3 vizinhança

In [18]:
cont3=[]
for i in range(1,size(viz2)[1],step=1)
    if viz2[i,2] in cont2
        name=viz2[i,4]
    else 
        name=viz2[i,2]
    end
    cont3=vcat(cont3,name)
end
cont3=unique(cont3);

In [19]:
cont(area,cont3,viz3)

,From#,From Name,To# - Circ#,To Name,Area
,Int64,String,String,String,String
1,822,PACATB-CE500,11525 # 01,PECEM2-CE500,3
2,822,PACATB-CE500,56100 # 01,JAGUA2-CE500,3
3,525,FORTA2-CE500,822 # 01,PACATB-CE500,3
4,516,QUIXAD-CE500,525 # 01,FORTA2-CE500,3
5,521,MILAGR-CE500,516 # 01,QUIXAD-CE500,3
6,11565,AÇU3---RN500,516 # 01,QUIXAD-CE500,2
7,11525,PECEM2-CE500,525 # 02,FORTA2-CE500,3
8,526,SOBRA3-CE500,11525 # 01,PECEM2-CE500,3
9,526,SOBRA3-CE500,11525 # 02,PECEM2-CE500,3


In [126]:
outOrganon(viz3)

'PACATB-CE500_PECEM2-CE500	'
BRANCH	822	11525	01
	END /
'PACATB-CE500_JAGUA2-CE500	'
BRANCH	822	56100	01
	END /
'FORTA2-CE500_PACATB-CE500	'
BRANCH	525	822	01
	END /
'QUIXAD-CE500_FORTA2-CE500	'
BRANCH	516	525	01
	END /
'MILAGR-CE500_QUIXAD-CE500	'
BRANCH	521	516	01
	END /
'AÇU3---RN500_QUIXAD-CE500	'
BRANCH	11565	516	01
	END /
'SOBRA3-CE500_PECEM2-CE500	'
BRANCH	526	11525	01
	END /
'PACATB-CE500_PECEM2-CE500	'
BRANCH	822	11525	01
	END /
'ACARA3-CE500_PECEM2-CE500	'
BRANCH	12902	11525	01
	END /
'TIANG2-CE500_SOBRA3-CE500	'
BRANCH	12905	526	01
	END /
'TIANG2-CE500_ACARA3-CE500	'
BRANCH	12905	12902	01
	END /
'TERES2-PI500_TIANG2-CE500	'
BRANCH	528	12905	01
	END /
'PARNAI-PI500_TIANG2-CE500	'
BRANCH	12901	12905	01
	END /
'JAGUA2-CE500_AÇU3---RN500	'
BRANCH	56100	11565	01
	END /
'PACATB-CE500_JAGUA2-CE500	'
BRANCH	822	56100	01
	END /
'PARNAI-PI500_ACARA3-CE500	'
BRANCH	12901	12902	01
	END /
'PARNAI-PI500_TIANG2-CE500	'
BRANCH	12901	12905	01
	END /
'PARNAI-PI500_BACABE-MA500	'
BRANCH	12901	

# Contingências totais

In [127]:
contingencias=vcat(viz1,viz2)
#contingencias=vcat(contingencias,viz3) #Caso deseja-se incluir 3 vizinhança. porém para a 3 melhor fazer na mão
sort!(contingencias,[1,3])

,From#,From Name,To# - Circ#,To Name,Area
,Int64,String,String,String,String
1,316,CSP----CE230,11225 # 01,PECEM2-CE230,3
2,316,CSP----CE230,11225 # 02,PECEM2-CE230,3
3,325,FORTA2-CE230,525 #01,FORTA2-CE500,3
4,325,FORTA2-CE230,525 #02,FORTA2-CE500,3
5,325,FORTA2-CE230,525 #03,FORTA2-CE500,3
6,325,FORTA2-CE230,525 #04,FORTA2-CE500,3
7,326,SOBRA3-CE230,526 #01,SOBRA3-CE500,3
8,326,SOBRA3-CE230,526 #02,SOBRA3-CE500,3
9,511,ENPECE-CE230,11225 # 01,PECEM2-CE230,3


In [128]:
#Remoção de elementos que se repetem. ex: se temos 2 buses de interesse, a msm linha é primeira vizinhança dos 2 buses
sum=0
for i in findall(nonunique(contingencias))
    delete!(contingencias,i-sum)
    sum+=1
end

In [129]:
outOrganon(contingencias)

'CSP----CE230_PECEM2-CE230	'
BRANCH	316	11225	01
	END /
'FORTA2-CE230_FORTA2-CE500	'
BRANCH	325	525	01
	END /
'SOBRA3-CE230_SOBRA3-CE500	'
BRANCH	326	526	01
	END /
'ENPECE-CE230_PECEM2-CE230	'
BRANCH	511	11225	01
	END /
'QUIXAD-CE500_FORTA2-CE500	'
BRANCH	516	525	01
	END /
'FORTA2-CE500_PACATB-CE500	'
BRANCH	525	822	01
	END /
'SOBRA3-CE500_PECEM2-CE500	'
BRANCH	526	11525	01
	END /
'PACATB-CE500_PECEM2-CE500	'
BRANCH	822	11525	01
	END /
'PACATB-CE500_JAGUA2-CE500	'
BRANCH	822	56100	01
	END /
'PACATB-CE230_PACATB-CE500	'
BRANCH	823	822	01
	END /
'PECEM2-CE230_PECEM2-CE500	'
BRANCH	11225	11525	01
	END /
'PECEM2-CE230_CFAISA-CE230	'
BRANCH	11225	55777	01
	END /
'PECEM2-CE230_CTAIBA-CE230	'
BRANCH	11225	55784	01
	END /
'PECEM2-CE230_TRAIRI-CE230	'
BRANCH	11225	55790	01
	END /
'SOBRALCER013_SOBRA3-CE500	'
BRANCH	12335	526	01
	END /
'PARNAI-PI500_ACARA3-CE500	'
BRANCH	12901	12902	01
	END /
'ACARA3-CE500_PECEM2-CE500	'
BRANCH	12902	11525	01
	END /
'ACARA3-CE500_ACARA3-CE230	'
BRANCH	12902	1290